In [132]:
import pandas as pd
import numpy as np
from ckiptagger import WS, POS, NER
import re

In [136]:
# ETL1
df =pd.read_excel('200801to202008falutTypes.xlsx')
colNameZh = [_.split('\n')[0] for _ in df.columns]
colNameEng = [_.split('\n')[1] for _ in df.columns]
df.columns = colNameEng

# filter NA
df = df[df['CE_Comment'].notna()] # 缺失內容非空 (22483, 9)
df = df[ df['CE_Comment'].apply(lambda x: pd.to_numeric(x, errors='coerce')).isna() ] # 缺失內容非單一數字 (22481, 9)
df = df[df['CE_Item2'].notna()] # 條非空 (15198, 9)
df.to_csv('data_ETL1.csv',index=False)
del df

In [172]:
# ETL2-1: retrive punctuation
import re
import pickle
df = pd.read_csv('data_ETL1.csv')

punctuation = list()
for i, s in enumerate(df['CE_Comment']):
    s = re.findall(r'[^\u4e00-\u9fff]+', s)
    s = ''.join(s)
    s = re.findall(r'[^\w]+', s)
    s = ''.join(s)
    punctuation = punctuation + list(s)
else:
    punctuation = set(punctuation)
    pickle.dump(punctuation, open('punctuation.pkl', 'wb'))

In [182]:
# ETL2-2: tokenized by ckip, delimiter by puncuation
from ckiptagger import WS, POS, NER
from ckiptagger import data_utils, construct_dictionary
path = "./ckip"
ws = WS(path)
# pos = POS(path)
# ner = NER(path)

df = pd.read_csv('data_ETL1.csv')
punctuation = pickle.load(open('punctuation.pkl', 'rb'))
word_s = ws(df['CE_Comment'],
            sentence_segmentation=True,
            segment_delimiter_set=punctuation)

In [184]:
df['token'] =['@'.join(_) for _ in word_s]

In [176]:
df.to_csv('data_ETL2.csv', index=False)